In [ ]:
from flask import Flask
from flask_cors import CORS, cross_origin
from draghts import DraughtsBoard, DraughtsPiece
from minimax import best_move
from flask import jsonify
from base import Player
import re

In [ ]:
regexp = "S([0-9])([0-9])_([A-Z]{1,2})"

In [ ]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

@app.route('/initial_position')
@cross_origin()
def initial_position():
    b = DraughtsBoard()
    pos = b.position
    pos = {f"S{square[0]}{square[1]}": piece.name for square, piece in pos.items() }
    return jsonify(pos)
    #return b.to_string()

@app.route('/move/<pos_string>')
@cross_origin()
def make_move(pos_string):
    print("in make_move")
    #pos_string = "S00_E&S01_E&S02_E&S03_E&S04_E&S05_E&S06_E"
    pos_list = pos_string.split("&")
    tomove = pos_list[0]
    pos_list = pos_list[1:]
    groups = [re.match(regexp, x).groups() for x in pos_list]
    pos = {(int(x[0]), int(x[1])): DraughtsPiece[x[2]] for x in groups}
    board = DraughtsBoard(pos, Player[tomove])
    move = best_move(board, 2)
    new_board = board.make_move(move)
    
    #print(position)
    ret = {f"S{square[0]}{square[1]}": piece.name for square, piece in new_board.position.items() }

    ret = jsonify(ret)
    return ret




In [ ]:
app.run()